# Jane Street: Neural Network Starter

I have used Jane Street: Neural Network Starter notebook. The purpose is to understand the source code.

In [1]:
import pandas as pd
import numpy as np

import janestreet

from tqdm.notebook import tqdm

import tensorflow as tf
tf.random.set_seed(73)
import tensorflow.keras.layers as layers

# Preprocessing

In [2]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv', nrows = 3)
features = [c for c in train.columns if 'feature' in c]

# Training

In [3]:
def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate):
    
    inp = tf.keras.layers.Input(shape = (num_columns, ))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)): 
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i+1])(x)    
        
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation('sigmoid')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                  loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = label_smoothing), 
                  metrics = tf.keras.metrics.AUC(name = 'AUC'), 
                 )
    
    return model

In [4]:
batch_size = 4096
hidden_units = [256, 512, 128]
dropout_rates = [0.10, 0.2, 0.3]
label_smoothing = 1e-2
learning_rate = 1e-3

# Load Models

In [5]:
num_models = 2

models = []
for i in range(num_models):
    clf = create_mlp(len(features), 1, hidden_units, dropout_rates, label_smoothing, learning_rate)
    clf.load_weights(f'../input/js-nn-models/JSModel_{i}.hdf5')
#     clf.load_weights(f'./JSModel_{i}.hdf5')
    models.append(clf)

IndexError: list index out of range

In [6]:
f_mean = np.load('../input/js-nn-models/f_mean.npy')
# f_mean = np.load('./f_mean.npy')

# Submitting

Just use two models to reduce running time.

In [7]:
env = janestreet.make_env()
env_iter = env.iter_test()

In [8]:
opt_th = 0.5
for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = 0.
        for clf in models:
            pred += clf(x_tt, training = False).numpy().item() / num_models
#         pred = models[0](x_tt, training = False).numpy().item()
        pred_df.action = np.where(pred >= opt_th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)